In [ ]:
# =============================================================================
# BƯỚC 0: CÀI ĐẶT VÀ IMPORT CÁC THƯ VIỆN CẦN THIẾT
# =============================================================================
!pip install -q noisereduce librosa

import os
import glob
import numpy as np
import librosa
import soundfile as sf
import noisereduce as nr
from tqdm.notebook import tqdm
from google.colab import drive
import concurrent.futures

# =============================================================================
# BƯỚC 1: KẾT NỐI GOOGLE DRIVE VÀ CẤU HÌNH
# =============================================================================
drive.mount('/content/drive')

# --- BẢNG ĐIỀU KHIỂN ---
SOURCE_FOLDER = "/content/drive/MyDrive/Tai_Lieu_NCKH/dataset_cough"
DESTINATION_FOLDER = "/content/drive/MyDrive/Tai_Lieu_NCKH/feature_data"
PROBLEM_FILES_LOG = "/content/drive/MyDrive/Tai_Lieu_NCKH/metadata1.txt"
SAMPLE_RATE = 16000
N_FFT = 2048
HOP_LENGTH = 512
N_MELS = 256
SILENCE_THRESHOLD_DB = 20
SEGMENT_LENGTH_S = 4
MIN_DURATION_S = 1
# ---------------------
os.makedirs(DESTINATION_FOLDER, exist_ok=True)
print(f"Dữ liệu nguồn sẽ được lấy từ: {SOURCE_FOLDER}")
print(f"Dữ liệu đặc trưng sẽ được lưu tại: {DESTINATION_FOLDER}")


# =============================================================================
# BƯỚC 2: ĐỊNH NGHĨA HÀM XỬ LÝ CHÍNH
# =============================================================================
def process_audio_file(file_path):
    # --- BỔ SUNG: KIỂM TRA FILE ĐÃ TỒN TẠI CHƯA ---
    original_filename = os.path.splitext(os.path.basename(file_path))[0]
    # Chỉ cần kiểm tra sự tồn tại của segment đầu tiên
    expected_first_segment_name = f"{original_filename}-seg0.npy"
    relative_path = os.path.dirname(os.path.relpath(file_path, SOURCE_FOLDER))
    final_dest_folder = os.path.join(DESTINATION_FOLDER, relative_path)
    expected_filepath = os.path.join(final_dest_folder, expected_first_segment_name)

    if os.path.exists(expected_filepath):
        # Nếu file đã tồn tại, trả về status mới và bỏ qua
        return file_path, "Skipped (Already Exists)", 0
    # --- KẾT THÚC BỔ SUNG ---

    try:
        y, sr = librosa.load(file_path, sr=SAMPLE_RATE, mono=True) # tải và chuẩn hóa tần số và định dạng âm thanh
        if len(y) / sr < MIN_DURATION_S:
            return file_path, "Skipped (Too Short)", 0 # Kiểm tra và bỏ qua các file âm thanh quá ngắ
        y = librosa.util.normalize(y) # Chuẩn hóa biên độ âm thanh
        y_denoised = nr.reduce_noise(y=y, sr=sr) # Giảm nhiễu 
        y_trimmed, _ = librosa.effects.trim(y_denoised, top_db=SILENCE_THRESHOLD_DB) # Cắt bỏ các khoảng lặng ở đầu và cuối càu file
        if len(y_trimmed) < 1:
            return file_path, "Skipped (Silent File)", 0 #Kiểm tra lại xem sau khi loại bỏ khoảng lặng thì file âm thanh có còn đủ độ dài ko

        segment_samples = int(SEGMENT_LENGTH_S * sr) # Tình toán số lượng mẫu được cắt ra
        segments_created = 0 # Khởi tạo biến đếm số lượng mẫu

        for i in range(0, len(y_trimmed), segment_samples):
            segment = y_trimmed[i:i + segment_samples] # Cắt các file am thanh ra thành cấc file có thời lượng bằng nhau(4 giây)
            if len(segment) < segment_samples: #Nếu độ dài sau khi cắt của file ngắn hơn thời gian quy định thì đệm thêm số 0 vào cuối
                padding_needed = segment_samples - len(segment) 
                segment = np.pad(segment, (0, padding_needed), 'constant')

            mels = librosa.feature.melspectrogram(y=segment, sr=sr, n_fft=N_FFT, hop_length=HOP_LENGTH, n_mels=N_MELS) # Tính toán Mel_spectrogram 
            mels_db = librosa.power_to_db(mels, ref=np.max) # Vẽ mel_spectrogram

            if not np.isfinite(mels_db).all(): #Kểm tra xem file có bị lỗi NaN hay Inf ko 
                continue

            segment_filename = f"{original_filename}-seg{segments_created}.npy" # Lưu các file vào thư mục output
            os.makedirs(final_dest_folder, exist_ok=True)
            save_path = os.path.join(final_dest_folder, segment_filename)
            np.save(save_path, mels_db)
            segments_created += 1

        if segments_created == 0:
            return file_path, "Skipped (No segments created)", 0
        return file_path, "Success", segments_created
    except Exception as e:
        return file_path, f"Error ({e})", 0

# =============================================================================
# BƯỚC 3: THỰC THI SONG SONG TRÊN NHIỀU NHÂN CPU
# =============================================================================
print("\nBắt đầu quá trình tiền xử lý song song...")

wav_files = glob.glob(os.path.join(SOURCE_FOLDER, '**', '*.wav'), recursive=True)
print(f"Tìm thấy {len(wav_files)} file .wav để xử lý.")

success_count, skipped_count, error_count, total_segments = 0, 0, 0, 0

with concurrent.futures.ProcessPoolExecutor() as executor:
    with open(PROBLEM_FILES_LOG, "w") as log_file:
        log_file.write("DANH SÁCH CÁC FILE GẶP SỰ CỐ KHI TIỀN XỬ LÝ\n" + "="*50 + "\n")

        futures = [executor.submit(process_audio_file, wav_file) for wav_file in wav_files]

        for future in tqdm(concurrent.futures.as_completed(futures), total=len(wav_files), desc="Processing files"):
            file_path, status, num_segments = future.result()

            if status == "Success":
                success_count += 1
                total_segments += num_segments
            elif "Skipped" in status:
                skipped_count += 1
                # Không cần ghi log cho file đã tồn tại để tránh làm đầy file log
                if "Already Exists" not in status:
                    log_file.write(f"{file_path}: {status}\n")
            else: # Error
                error_count += 1
                log_file.write(f"{file_path}: {status}\n")

# In báo cáo tổng kết
print("\n" + "="*50)
print("QUÁ TRÌNH TIỀN XỬ LÝ HOÀN TẤT!")
print(f"Thành công (mới): {success_count} files")
print(f"Bị bỏ qua (ngắn/lỗi/đã tồn tại): {skipped_count} files")
print(f"Bị lỗi: {error_count} files")
print(f"Tổng số file đặc trưng (.npy) mới đã tạo: {total_segments}")
print(f"Danh sách các file bị bỏ qua/lỗi được ghi tại: {PROBLEM_FILES_LOG}")
print("="*50)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dữ liệu nguồn sẽ được lấy từ: /content/drive/MyDrive/Tai_Lieu_NCKH/dataset_cough
Dữ liệu đặc trưng sẽ được lưu tại: /content/drive/MyDrive/Tai_Lieu_NCKH/feature_data

Bắt đầu quá trình tiền xử lý song song...
Tìm thấy 27837 file .wav để xử lý.


Processing files:   0%|          | 0/27837 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
# =============================================================================
# BƯỚC 0: CÀI ĐẶT, IMPORT VÀ XÁC THỰC
# =============================================================================
!pip install -q kaggle

import os
import json
import shutil
import subprocess # <-- Thêm thư viện này
from google.colab import drive, files

drive.mount('/content/drive')

print("Vui lòng tải lên file kaggle.json của bạn:")
try:
    if os.path.exists('/content/kaggle.json'):
        os.remove('/content/kaggle.json')
    uploaded = files.upload()
    if 'kaggle.json' in uploaded:
        os.makedirs('/root/.kaggle', exist_ok=True)
        shutil.move('/content/kaggle.json', '/root/.kaggle/kaggle.json')
        os.chmod('/root/.kaggle/kaggle.json', 600)
        print("\nXác thực Kaggle thành công!")
    else:
        raise FileNotFoundError("Không tìm thấy file kaggle.json.")
except Exception as e:
    print(f"\nĐã xảy ra lỗi trong quá trình xác thực: {e}")

# =============================================================================
# BƯỚC 1: CẤU HÌNH
# =============================================================================
SOURCE_DRIVE_FOLDER = "/content/drive/MyDrive/Tai_Lieu_NCKH/feature_data"
KAGGLE_DATASET_TITLE = "NGT-spectrogram"
KAGGLE_DATASET_ID = "NGT-spectrogram-id" # Sửa ID để tránh trùng lặp
TEMP_ZIP_FILENAME = "dataset_for_kaggle.zip"
TEMP_ZIP_PATH = f"/content/{TEMP_ZIP_FILENAME}"

# =============================================================================
# BƯỚC 2: NÉN DỮ LIỆU TỪ GOOGLE DRIVE
# =============================================================================
print(f"\nBắt đầu nén thư mục: {SOURCE_DRIVE_FOLDER}...")
try:
    shutil.make_archive(base_name='/content/dataset_for_kaggle', format='zip', root_dir=SOURCE_DRIVE_FOLDER)
    if os.path.exists(TEMP_ZIP_PATH):
        print(f"Nén dữ liệu thành công! File được lưu tại: {TEMP_ZIP_PATH}")
    else:
        raise FileNotFoundError("Không thể tạo file zip.")

    # =========================================================================
    # BƯỚC 3: TẠO VÀ TẢI DATASET LÊN KAGGLE (PHIÊN BẢN SỬA LỖI)
    # =========================================================================
    print("\nBắt đầu tạo dataset trên Kaggle...")

    # 1. Tạo một thư mục tạm thời, sạch sẽ để tải lên
    UPLOAD_DIR = "/content/upload_to_kaggle"
    if os.path.exists(UPLOAD_DIR):
        shutil.rmtree(UPLOAD_DIR) # Xóa thư mục cũ nếu có
    os.makedirs(UPLOAD_DIR)
    print(f"Đã tạo thư mục tải lên tạm thời: {UPLOAD_DIR}")

    # 2. Di chuyển file zip đã nén vào thư mục tải lên
    shutil.move(TEMP_ZIP_PATH, os.path.join(UPLOAD_DIR, TEMP_ZIP_FILENAME))

    # 3. Khởi tạo và cập nhật file metadata trong thư mục tải lên
    !kaggle datasets init -p {UPLOAD_DIR}
    metadata_file = os.path.join(UPLOAD_DIR, 'dataset-metadata.json')

    with open("/root/.kaggle/kaggle.json", "r") as f:
        kaggle_username = json.load(f)['username']
    dataset_slug = f"{kaggle_username}/{KAGGLE_DATASET_ID}"

    with open(metadata_file, 'r') as f:
        metadata = json.load(f)
    metadata['title'] = KAGGLE_DATASET_TITLE
    metadata['id'] = dataset_slug
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=4)
    print("Metadata đã được cập nhật.")


    # 4. Tải lên thư mục đã chuẩn bị (KHÔNG DÙNG --dir-mode zip)
    print("\nĐang tải dữ liệu lên Kaggle. Quá trình này có thể mất vài phút...")
    command = ["kaggle", "datasets", "create", "-p", UPLOAD_DIR] # Trỏ đến thư mục mới
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, encoding='utf-8')

    print("\n--- Bắt đầu Log từ Kaggle API ---")
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())
    return_code = process.poll()
    print("--- Kết thúc Log từ Kaggle API ---")


    if return_code == 0:
        print("\n" + "="*50)
        print("QUÁ TRÌNH HOÀN TẤT!")
        print(f"Bạn có thể truy cập dataset tại: https://www.kaggle.com/datasets/{dataset_slug}")
        print("="*50)
    else:
        print("\n" + "!"*50)
        print(f"LỖI: Quá trình tải lên Kaggle thất bại với mã lỗi: {return_code}.")
        print("Vui lòng xem lại log bên trên để biết chi tiết lỗi.")
        print("!"*50)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Vui lòng tải lên file kaggle.json của bạn:


Saving kaggle.json to kaggle.json

Xác thực Kaggle thành công!

Bắt đầu nén thư mục: /content/drive/MyDrive/Tai_Lieu_NCKH/feature_data...
Nén dữ liệu thành công! File được lưu tại: /content/dataset_for_kaggle.zip

Bắt đầu tạo dataset trên Kaggle...
Data package template written to: /content/dataset-metadata.json
Metadata đã được cập nhật.

Đang tải dữ liệu lên Kaggle. Quá trình này có thể mất vài phút...

--- Bắt đầu Log từ Kaggle API ---
Starting upload for file .config.zip

0%|          | 0.00/9.30k [00:00<?, ?B/s]
100%|██████████| 9.30k/9.30k [00:00<00:00, 40.1kB/s]
Upload successful: .config.zip (9KB)
[Errno 95] Operation not supported: '/content/drive/.shortcut-targets-by-id/1jGnda6fl0n2R_DOeLcmoZA9FpmFZWDa9/Tai_Lieu_NCKH/BÁO CÁO CHẨN ĐOÁN TIẾNG HO.gdoc'
--- Kết thúc Log từ Kaggle API ---

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
LỖI: Quá trình tải lên Kaggle thất bại với mã lỗi: 1.
Vui lòng xem lại log bên trên để biết chi tiết lỗi.
Một số lỗi thường gặp: ID data

In [9]:
# =========================================================================
# BƯỚC 3: TẠO VÀ TẢI DATASET LÊN KAGGLE (PHIÊN BẢN SỬA LỖI)
# =========================================================================
print("\nBắt đầu tạo dataset trên Kaggle...")

# 1. Tạo một thư mục tạm thời, sạch sẽ để tải lên
UPLOAD_DIR = "/content/upload_to_kaggle"
if os.path.exists(UPLOAD_DIR):
    shutil.rmtree(UPLOAD_DIR) # Xóa thư mục cũ nếu có
os.makedirs(UPLOAD_DIR)
print(f"Đã tạo thư mục tải lên tạm thời: {UPLOAD_DIR}")

# 2. Di chuyển file zip đã nén vào thư mục tải lên
shutil.move(TEMP_ZIP_PATH, os.path.join(UPLOAD_DIR, TEMP_ZIP_FILENAME))

# 3. Khởi tạo và cập nhật file metadata trong thư mục tải lên
!kaggle datasets init -p {UPLOAD_DIR}
metadata_file = os.path.join(UPLOAD_DIR, 'dataset-metadata.json')

with open("/root/.kaggle/kaggle.json", "r") as f:
    kaggle_username = json.load(f)['username']
dataset_slug = f"{kaggle_username}/{KAGGLE_DATASET_ID}"

with open(metadata_file, 'r') as f:
    metadata = json.load(f)
metadata['title'] = KAGGLE_DATASET_TITLE
metadata['id'] = dataset_slug
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=4)
print("Metadata đã được cập nhật.")


# 4. Tải lên thư mục đã chuẩn bị (KHÔNG DÙNG --dir-mode zip)
print("\nĐang tải dữ liệu lên Kaggle. Quá trình này có thể mất vài phút...")
command = ["kaggle", "datasets", "create", "-p", UPLOAD_DIR] # Trỏ đến thư mục mới
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, encoding='utf-8')

print("\n--- Bắt đầu Log từ Kaggle API ---")
while True:
    output = process.stdout.readline()
    if output == '' and process.poll() is not None:
        break
    if output:
        print(output.strip())
return_code = process.poll()
print("--- Kết thúc Log từ Kaggle API ---")


if return_code == 0:
    print("\n" + "="*50)
    print("QUÁ TRÌNH HOÀN TẤT!")
    print(f"Bạn có thể truy cập dataset tại: https://www.kaggle.com/datasets/{dataset_slug}")
    print("="*50)
else:
    print("\n" + "!"*50)
    print(f"LỖI: Quá trình tải lên Kaggle thất bại với mã lỗi: {return_code}.")
    print("Vui lòng xem lại log bên trên để biết chi tiết lỗi.")
    print("!"*50)


Bắt đầu tạo dataset trên Kaggle...
Đã tạo thư mục tải lên tạm thời: /content/upload_to_kaggle
Data package template written to: /content/upload_to_kaggle/dataset-metadata.json
Metadata đã được cập nhật.

Đang tải dữ liệu lên Kaggle. Quá trình này có thể mất vài phút...

--- Bắt đầu Log từ Kaggle API ---
Starting upload for file dataset_for_kaggle.zip

0%|          | 0.00/786M [00:00<?, ?B/s]
1%|          | 4.17M/786M [00:00<00:18, 43.7MB/s]
2%|▏         | 12.3M/786M [00:00<00:11, 67.9MB/s]
3%|▎         | 23.3M/786M [00:00<00:08, 89.6MB/s]
5%|▍         | 36.2M/786M [00:00<00:07, 107MB/s]
6%|▌         | 46.8M/786M [00:00<00:07, 109MB/s]
8%|▊         | 59.6M/786M [00:00<00:06, 117MB/s]
9%|▉         | 71.8M/786M [00:00<00:06, 121MB/s]
11%|█         | 83.4M/786M [00:00<00:06, 120MB/s]
12%|█▏        | 94.8M/786M [00:00<00:06, 116MB/s]
14%|█▎        | 107M/786M [00:01<00:05, 119MB/s]
15%|█▌        | 118M/786M [00:01<00:06, 114MB/s]
17%|█▋        | 130M/786M [00:01<00:05, 118MB/s]
18%|█▊     